In this project, I built a language identification classifier that distinguishes between six languages:

- [Hausa](https://en.wikipedia.org/wiki/Hausa_language)
- [Indonesian](https://en.wikipedia.org/wiki/Indonesian_language)
- [Manobo](https://en.wikipedia.org/wiki/Manobo_languages)
- [Nahuatl](https://en.wikipedia.org/wiki/Nahuatl)
- [Swahili](https://en.wikipedia.org/wiki/Swahili_language)
- [Tagalog](https://en.wikipedia.org/wiki/Tagalog_language)

Some languages can be distinguished easily, because they use different scripts. These six languages, however, use the same ([Latin](https://en.wikipedia.org/wiki/Latin_script)) script with minimal [diacritics](https://en.wikipedia.org/wiki/Diacritic) so it is difficult to hand-craft classifiers based on the presence or absence of particular characters. Indeed, unless you have linguistic training or familiarity with the languages, it is difficult to tell them apart.

How can they be distinguished? A naïve approach is to use word counts as unigram features. However, the number of possible words in a large corpus of five languages is vast. It is essential to look at something smaller — characters.

Even though the six languages use roughly the same characters, the relative frequencies of these characters vary greatly. Thus, using characters as features (unigram character models) is appealing (and fairly effective). It is also true that languages vary in their *phonotactics*, the way in which consonants and vowels combine in sequence. Thus, looking at character ngrams (for small values of $n$) is also appealing (and effective). Note, however, that as the value of $n$ increases, this approach runs into the same problem as the word unigram model (sparcity). In this scenario, the model is likely to overfit.

Various kinds of classifiers can be used for this application. NB classifiers, for example, are quite effective. However, inference is slow and performance, given the same training set, is likely to be worse than other options. Simple logistic regression cannot be used because this is an n-way (multinomial) classification problem. Multinomial Logistic Regression (Softmax Regression) is a good fit.

## Requirements

You will perform the following tasks:

1. Implement a training loop for Multinomial Logistic Regression.
2. Implement inference for Multinomal Logistic Regression
3. Determine the optimal order of $n$ for ngrams for MNLR trained on the training set.
4. Calculate and display a confusion matrix for a trigram model evaluated on the test set.
5. Inspect the feature weights, and display the most predictive features for each language.

## How to Use Jupyter Notebooks for Our Assignment
Throughout our first assignment, we'll utilize Jupyter Notebooks to walk you through concepts, let you implement them, and also allow you to experiment on your own. By the end of this assignment, you'll not only understand word embeddings more deeply but will also become familiar with a powerful tool used extensively in the data science community.

### Types of Cells:
**Markdown Cells**: These cells (like the one you're reading now) are utilized to write text, frame explanations, embed images, or even formulate equations. They make our notebook more explanatory and structured.

**Code Cells**: This is where the action happens. In these cells, you'll write and execute Python code. They will play a critical role in our exercises as you experiment with word embeddings.

*Warning*: Refrain from rearranging, adding, or deleting any cells.

### Runtime Volatility
As you navigate and execute the cells within this Jupyter notebook, it's crucial to understand that the runtime environment is volatile. In simpler terms:

*   If you restart the notebook or experience a disconnection, all your in-memory data and variables will be lost.
*   While the code and markdown cells will remain, the outputs from code cells will need to be regenerated by rerunning them.

Therefore, if you're working on a task over an extended period or with large datasets, remember to save your results and progress frequently to avoid potential data loss.


### Running this Notebook on Google Colab
Google Colab is a free Jupyter notebook environment that requires no setup and runs entirely in the cloud. To run this notebook on Colab:

1.   Save a copy of this notebook on your **Google Drive**.
2.   Open the notebook in Google Colab.
3.   In Google Colab, you can execute each cell using the play button (or use the keyboard shortcuts mentioned above).

**Tip**: Google Colab may automatically disconnect after a certain period of inactivity. Keep this in mind, especially when running longer tasks.

**Note**: Google Colab provides free access to GPUs and TPUs, which might be useful for later assignments.

## Important: Library Restrictions

**You must implement multinomial logistic regression from scratch using only the provided imports.**

You are **NOT allowed** to use:
- PyTorch, TensorFlow, or any other deep learning frameworks
- Scikit-learn or other machine learning libraries  
- Any external optimization libraries

You may only use the imported libraries: `csv`, `random`, `numpy`, and `collections.Counter`.

## Imports
Do not change.

In [1]:
import csv
import random

import numpy as np
import numpy.typing as npt
import numpy.testing as testing

from collections import Counter

## Helper Functions
### Understanding Data Structures

Throughout this assignment, you'll work with these key data structures:

1. **Raw observations**: `list[tuple[str, str]]` - List of (language_name, document_text) pairs
2. **Processed observations**: `list[tuple[np.ndarray, np.ndarray]]` - List of (one_hot_label, feature_vector) pairs
3. **Weight matrix W**: `np.ndarray` shape `[K, N]` where K=number of languages, N=number of features
   - **Important**: Column 0 contains bias terms, so remember to prepend 1 to feature vectors
4. **Feature map**: `dict[str, int]` - Maps n-gram strings to feature indices
5. **Language map**: `dict[str, np.ndarray]` - Maps language names to one-hot vectors

### Metrics for Binary Classification

In [2]:
def precision(tp: int, fp: int) -> float:
    """Computes the precision, given true positives and false positives."""
    return tp / (tp + fp)


def recall(tp: int, fn: int) -> float:
    """Computes the recall, given the true positives and false negatives."""
    return tp / (tp + fn)


def f_measure(beta: float, tp: int, fp: int, fn: int) -> float:
    """Computes the F-measure for a given beta, true positives, false positives, and false negatives."""
    return (
        (1 + beta**2)
        * (precision(tp, fp) * recall(tp, fn))
        / (beta**2 * precision(tp, fp) + recall(tp, fn))  # ← FIXED: + instead of *
    )


def f1(tp: int, fp: int, fn: int) -> float:
    """Computes the F1 measure for a given TP, FP, and FN."""
    return f_measure(1, tp, fp, fn)

### Micro-Averaged Metrics

In [3]:
def micro_precision(tp: "dict[str, int]", fp: "dict[str, int]") -> float:
    """Computes micro-averaged precision."""
    tp_sum = sum(tp.values())
    fp_sum = sum(fp.values())
    return tp_sum / (tp_sum + fp_sum)


def micro_recall(tp: "dict[str, int]", fn: "dict[str, int]") -> float:
    """Computes micro-averaged recall."""
    tp_sum = sum(tp.values())
    fn_sum = sum(fn.values())
    return tp_sum / (tp_sum + fn_sum)


def micro_f1(tp: "dict[str, int]", fp: "dict[str, int]", fn: "dict[str, int]") -> float:
    """Computes micro-averaged F1."""
    mp = micro_precision(tp, fp)
    mr = micro_recall(tp, fn)
    return 2 * (mp * mr) / (mp + mr)

### Macro-Averaged Metrics

In [4]:
def macro_precision(tp: "dict[str, int]", fp: "dict[str, int]") -> float:
    """Computes macro-averaged precision."""
    n = len(tp)
    return (1 / n) * sum([precision(tp[c], fp[c]) for c in tp.keys()])


def macro_recall(tp: "dict[str, int]", fn: "dict[str, int]") -> float:
    """Computes macro-averaged recall."""
    n = len(tp)
    return (1 / n) * sum([recall(tp[c], fn[c]) for c in tp.keys()])


def macro_f1(tp: "dict[str, int]", fp: "dict[str, int]", fn: "dict[str, int]") -> float:
    """Computes macro-averaged F1."""
    n = len(tp)
    return (
        2
        * (macro_precision(tp, fp) * macro_recall(tp, fn))
        / (macro_precision(tp, fp) + macro_recall(tp, fn))
    )

## Preprocess Data

### Loading data from files

We first need to load data from the provided TSV files. Each file is two columns, the language of the document and the document text, separated by a tab (`\t`) character. We load this data into a list of tuples, to maintain the coupling between each document and its label.

In [5]:
def load_data(data_filename: str) -> list[tuple[str, str]]:
    with open(data_filename, 'r', errors='backslashreplace', encoding='utf-8') as f_in:
        language_document_tuples = [(line.split("\t")[0], line.split("\t")[1]) for line in f_in.read().split("\n") if line != '']
    return language_document_tuples

### Feature Extraction

We will use ngrams as features, so we need to be able to extract them.

In [6]:
def extract_ngrams(x: str, n=3) -> "list[str]":
    """Given a string, return all character ngrams of order `n`."""
    return ["".join(s) for s in (zip(*[x[i:] for i in range(n)]))]

### Language Codes to One-Hot Vectors
And we need a function to convert a language code into a **one-hot vector** (called $\mathbf{y}$).

In [7]:
def to_onehot_vector(lang: str, langs: list[str]) -> np.ndarray:
    y = np.zeros(len(langs))
    y[langs.index(lang)] = 1
    return y

We need to be able to convert `dict`s of ngram counts to vectors of ngram counts (using a map from ngrams to dimensions of the vectors).

In [8]:
def vectorize_ngrams(counter: dict[str, int], feature_map: dict[str, int]) -> np.ndarray:
    """
    Given a dict of ngram counts and a map from features to indices, returns a vector of ngram counts.

    Parameters
    ----------
    counter : dict
        Counter/dict of ngram counts
    feature_map : dict
        Map from ngrams to indices

    Returns
    -------
    np.ndarray
        A vector of ngram counts
    """
    feature_vector = np.zeros(len(feature_map))
    for ngram, count in counter.items():
        if ngram in feature_map:
            feature_vector[feature_map[ngram]] = count
    return feature_vector

And putting together the conversion from text into ngrams, and vectorizing the ngrams:

In [9]:
def vectorize_document(document: str, feature_map: dict[str, int], ngram_length: int) -> np.ndarray:
    document_ngrams = extract_ngrams(document, ngram_length)
    vector = vectorize_ngrams(Counter(document_ngrams), feature_map)
    return vector

We need separate functions for preprocessing the training observations and the dev/test observations. The former function must return a map from language names to labels as one-hot vectors as well as a map from features (ngrams) to indices of vector dimensions.

In [10]:
def preprocess_training_observations(
    training_observations: list[tuple[str, str]], n: int = 1
) -> tuple[list[tuple[np.ndarray, np.ndarray]], dict[str, np.ndarray], dict[str, int]]:
    langs = set()
    features = set()
    obs = []

    for lang, doc in training_observations:
        langs.add(lang)
        ngrams = extract_ngrams(doc, n)
        features = features | set(ngrams)
        obs.append((lang, ngrams))
    feature_map = {feature: idx for idx, feature in enumerate(sorted(features))}
    lang_list = list(sorted(langs))
    lang_map = {lang: to_onehot_vector(lang, lang_list) for lang in lang_list}

    obs = [
        (lang_map[lang], vectorize_ngrams(Counter(ngrams), feature_map)) for (lang, ngrams) in obs
    ]

    # print("Obs:", obs[0])
    # print("Lang Map:", lang_map, "\n")

    # print("Training observations Lang:", training_observations[0][0], "\n")
    # print("Training observations doc:", training_observations[0][1])

    print(f"{len(obs)} training observations.")
    return obs, lang_map, feature_map

The function for preprocessing test observations (and dev observations) takes the feature map and the language map as arguments and returns only the list of labeled observations.

In [11]:
def preprocess_test_observations(
    test_observations, feature_map: dict[str, int], lang_map: dict[str, np.ndarray], n: int = 1
) -> tuple[list[np.ndarray], list[np.ndarray]]:
    obs = []

    for i, (lang, doc) in enumerate(test_observations):
        vectorized_doc = vectorize_document(doc, feature_map, ngram_length=n)
        try:
            obs.append((lang_map[lang], vectorized_doc))
        except KeyError:
            print(f"Unkown language {lang} at index {i}. Known languages are: {lang_map.keys()}")
    print(f"{len(obs)} test observations.")
    return obs

### Classification Function

We also need to define the softmax function.

Softmax is technically

$$\text{softmax}(z_i) = \frac{e^{z_i}}{\sum^K_{j=1} e^{z_j}}$$

However, if implemented naïvely, this is not numerically stable. Instead, we use:

$$\text{softmax}(z_i) = \frac{e^{z_i - \text{max}(z)}}{\sum^K_{j=1} e^{z_j - \text{max}(z)}}$$


In [12]:
def softmax(z: npt.ArrayLike) -> npt.ArrayLike:
    """Compute the softmax of a vector `z`"""
    # exp(z) can get very large. For numerical stability, we subtract a vector of very large values (np.max(z)) from z.
    return np.exp(z - np.max(z)) / np.exp(z - np.max(z)).sum()

## Training the Classifier

### Compute the gradient

The formula for computing one element in our gradient is as follows (the partial derivitive of the negative log likelihood loss):

$$\frac{\partial L_{CE}}{\partial \mathbf{w}_{k,i}}=-(\mathbf{y}_k-\hat{\mathrm{y}}_k)\mathbf{x}_i$$

where $k$ is the **class** (rows of the matrix $\mathbf{w}$) and $i$ corresponds the the feature (columns of the matrix $\mathbf{w}$).

We will define a function `grad` for computing the whole gradient, a $K \times N$ matrix.

**This is the first piece of code that you'll write.**

In [13]:
def grad(W: np.ndarray, y: np.ndarray, x: np.ndarray) -> np.ndarray:
    """
    Caculates the gradient of the negative log liklihood loss, a [K * N] matrix, with respect to W.

    Parameters
    ----------
    W : np.ndarray
        A matrix of of weights, expressed as a [K * N] matrix where K is the number of classes and N is the number of features.
    y : np.ndarray
       The true label of the observation, expressed as a one-hot encoded vector of shape [K].
    x : np.ndarray
        A vector of features.

    Returns
    -------
    np.ndarray
        The gradient of the loss with respect to W.
    """

    # Grad = -(Yk - Y_hat)Xi
    # Yk = rows of W
    # Y_hat = cols of W
    # Xi = a single feature vector x

    # z = w . x + b
    x = x.flatten() # flatten x to turn it to a matrix of 1-dimensional matrix for multiplication
    # print("Gradient: x flatten:", x, "\n")
    z = np.matmul(W, x)

    # print("grad() z: ", z, "\n")

    # Y_hat = softmax(z)
    Y_hat = softmax(z)

    # error = (Yk - Y_hat)
    error = y - Y_hat

    # print("Error:", error)

    # grad = -(Yk - Y_hat)Xi
    grad = -np.outer(error, x)

    return grad




### Training Loop

Iterate over the observations in the training set $n$ times (in random order). For each item, compute the one-hot vector $\mathbf{y}$ and the probability distribution $\hat{\mathbf{y}}$. Use these values to compute the gradient. Update the parameters based on the gradient. At the end of each epoch (pass through the training data), compute the true positives, false positives, and false negatives for each target class based on the current weights, and report micro-averaged precision and recall.

**Note**: The way that we evaluated the training loop depended on an in-place shuffle of the training data, which, while not technically wrong, is maybe not the most intuitive way to do it. When shuffling your data, please either use a shuffling method that does not change the observations parameter in-place. In particular, please use `random.sample` like:
```
shuffled_observations = random.sample(observations, len(observations))
```

How you report the metrics is up to you — we will not look at what you output to STDOUT — but it is important that you do this. **Otherwise, you will not be able to determine whether your model is training.**

You can also output the loss at each step (very noisy!), each epoch, or run the classifier on the dev set and report the metrics at the end of each epoch.

Remember that the 0th column in $W$ contains the biases. You will have to insert a $1$ at the beginning of the feature vector $x$ in order to accomodate this.

### Important Note About Bias Terms

Your weight matrix W will have shape [K, N+1] where the first column (index 0) contains bias terms for each class. When computing gradients and making predictions, you must:

1. **Prepend 1 to your feature vectors** to account for the bias term
2. **Initialize W with an extra column** for bias terms
3. **Remember this when vectorizing documents for prediction**

Example: If your feature vector x has shape [1000], prepend 1 to make it shape [1001] before matrix multiplication with W.

In [14]:
def train(observations: list[tuple[np.ndarray, np.ndarray]], eta: float, epochs: int = 1) -> np.ndarray:
    """
    Given a set of observations, returns a trained multinomial LR (softmax regression) model.

    Parameters
    ----------
    observations : list[tuple[np.ndarray, np.ndarray]]
        A list of tuples, where each tuple contains (y, x):
        - y: one-hot vector encoding the ground truth language label (shape [K])
        - x: vector of features (shape [N])
        Note: Remember to add bias term (prepend 1 to x) when computing gradients.
    eta : float
        Learning rate for gradient descent.
    epochs : int
        The number of epochs to train the model.

    Returns
    -------
    np.ndarray
        The trained model as a [K * N] weight matrix, where column 0 contains bias terms.
    """

    # Shape [K] = 6 (6 languages)
    # y: (swahili, manobo, nahuatl, hausa, tagalog, indomesian) e.g [1 0 0 0 0 0]
    # x: vector of features (Unatakiwa ukumbuke kuwa ulikuwa mtumwa katika nchi ya Misri; kwa hiyo nakuagiza kutii amri hii.)

    # onehot_vector e.g [0, 0, 0, 0, 1, 0] shape K
    onehot_vector = observations[0][0]

    # features_vector e.g [0. 0, 0, 1, 0, 1, 0, 12, 14, 18, 45] shape N
    features_vector = observations[0][1]

    # initialize W with an extra column for bias terms (W = K X N)
    weights_matrix = np.zeros((onehot_vector.shape[0], features_vector.shape[0]+1))


    for step in range(epochs):
        shuffled_observations = random.sample(observations, len(observations))

        for observation in shuffled_observations:
            onehot_vec = observation[0]
            features_vec = observation[1]

            biased_features_vec = np.concatenate(([1], features_vec)) # prepend 1 to features vector to account for bias terms

            # print("One hot vector", onehot_vec, "\n")
            # print("Feature vector", features_vec, "\n")

            gradient = grad(weights_matrix, onehot_vec, biased_features_vec)

            weights_matrix -= eta * gradient

    return weights_matrix

## Classification

The classification function is very simple.

In [15]:
def classify(W: np.ndarray, x: np.ndarray) -> np.intp:
    """
    Return the index of the hypothesized language.

    Parameters
    ----------
    W : np.ndarray
        Weight matrix (one row for each category/language, on column for each feature)
    x : np.ndarray
        Vector of real-valuled features

    Returns
    -------
    intp
        The index of the hypothesized language.
    """

    # Z = W . X + b
    # biased_x = X + b
    biased_x = np.concatenate(([1], x))

    # z = W . (x + b)
    z = np.dot(W, biased_x)

    # print("Argmax ", np.argmax(z))

    return np.argmax(z)

## Evaluate the Classifier

Then, a function to train and evaluate the classifier.

In [16]:
def evaluate(train_set, test_set, eta, epochs=3):
    """Trains and evaluates a model."""
    print("Training model")
    W = train(train_set, eta, epochs=epochs)
    print("\nCLASSIFY")
    tp, fp, fn = Counter(), Counter(), Counter()
    for ref_lang_vec, x in test_set:
        # print("Ref lang vec", ref_lang_vec)
        ref_lang = np.argmax(ref_lang_vec)
        # print("Ref lang ", ref_lang)

        hyp_lang = classify(W, x)
        # print("hyp_lang lang ", hyp_lang)
        if hyp_lang == ref_lang:
            tp[ref_lang] += 1
        else:
            fp[hyp_lang] += 1
            fn[ref_lang] += 1
    # Print metrics

    test_macro_f1 = macro_f1(tp, fp, fn)
    test_micro_f1 = micro_f1(tp, fp, fn)
    print(f"macro-averaged F1:\t\t{test_macro_f1:.3f}")
    print(f"micro-averaged F1:\t\t{test_micro_f1:.3f}")
    return test_macro_f1, test_micro_f1

### Putting it all together

Load the data:

In [17]:

train_observations = load_data('train.tsv')
test_observations = load_data('test.tsv')

### Inspecting the data

Before we actually train the classifier, it's important to look at your data, and check that any assumptions you're making about it are justified. It's always useful at this point to check basic things, like:
- How many instances of train and test data do you have?
- What labels are in your data, and do those match between the train and test splits?
- What is the class balance (i.e. how many instances of each class) in your dataset? Is it balanced or unbalanced?

Output a dictionary for the train and test data that maps each label to the count of instances that have that label, e.g.:
```
{"hausa": 4000, "indonesian":...}
```

Your dictionary should be sorted in descending order of occurrence of languages.

In [18]:
# this cell's output will be used for test 1.1
# your code here - train set
langs = []
for obs in train_observations:
    langs.append(obs[0])

train_dict = dict(Counter(langs))
sorted_dict = sorted(train_dict.items(), key=lambda item:item[1], reverse=True)

print(dict(sorted_dict))


{'swahili': 411, 'tagalog': 391, 'manobo': 114, 'hausa': 98, 'nahuatl': 93, 'indonesian': 93}


In [19]:
# this cell's output will be used for test 1.2
# your code here - test set
langs = []
for obs in test_observations:
    langs.append(obs[0])

train_dict = dict(Counter(langs))
sorted_dict = sorted(train_dict.items(), key=lambda item:item[1], reverse=True)

print(dict(sorted_dict))

{'swahili': 148, 'tagalog': 121, 'nahuatl': 37, 'manobo': 34, 'indonesian': 32, 'hausa': 28}


### Set hyperparameters and parameters

Before training the model, we have to set the learning rate $\eta$ and the order of the ngrams used in feature extraction.

In [20]:
eta = 0.0005  # Do not change this.
epochs = 4  # Do not change this.

In [21]:
order_of_ngrams = 2  # Change this.

### Running our training and evaluation loops

Now train your classifier, and evaluate it on the test set. Vary the number of ngrams, and observe how it changes train and test F1.

In [22]:
train_set, lang_map, feature_map = preprocess_training_observations(
    train_observations, n=order_of_ngrams
)
test_set = preprocess_test_observations(test_observations, feature_map, lang_map, n=order_of_ngrams)

# print("Train set", train_set[1], "\n")

# print("Test set", test_set[1])

random.seed(27)
test_macro_f1, test_micro_f1 = evaluate(train_set, test_set, eta, epochs=epochs)

1200 training observations.
400 test observations.
Training model

CLASSIFY
macro-averaged F1:		0.976
micro-averaged F1:		0.985


Print the tuple of the best values of `(macro_f1, micro_f1)` for the model evaluated on your test set while varying the order of the ngrams. What value of n produced the best result? Why might that be?

In [23]:
# the output of this cell will be used for test 2.1
print((test_macro_f1, test_micro_f1))

(0.9757823365261366, 0.985)


## Inspecting Classification Results

We've trained our classifier, and your final F1 should be pretty close to 1.0. Great job! But what does that mean for the languages you're actually classifying? Let's rewrite our evaluation code to allow us to look at our results instance-by-instance. For this, we're going to examine the results of a re-trained trigram classifier, trained for one epoch.

In [24]:
# Re-training a trigram classifier. Do not change this.

INSPECTION_NGRAMS = 3

train_set, lang_map, feature_map = preprocess_training_observations(
    train_observations, n=INSPECTION_NGRAMS
)
test_set = preprocess_test_observations(
    test_observations, feature_map, lang_map, n=INSPECTION_NGRAMS
)

random.seed(27)
W_inspect = train(train_set, eta, epochs=1)

## evaluate it as before. Check that this looks the same!
tp, fp, fn = Counter(), Counter(), Counter()
for ref_lang_vec, x in test_set:
    ref_lang = np.argmax(ref_lang_vec)

    hyp_lang = classify(W_inspect, x)
    if hyp_lang == ref_lang:
        tp[ref_lang] += 1
    else:
        fp[hyp_lang] += 1
        fn[ref_lang] += 1
# Print metrics

print(f"macro-averaged F1:\t\t{macro_f1(tp, fp, fn):.3f}")
print(f"micro-averaged F1:\t\t{micro_f1(tp, fp, fn):.3f}")

1200 training observations.
400 test observations.
macro-averaged F1:		0.946
micro-averaged F1:		0.960


### Writing a prediction function

Write a function that takes a list of observations, and produces a list of either class indices, or class names based on a parameter. This will allow you both to look at individual results from evaluating on an existing set of data (like the test set), but also for you to evaluate your classifier on new data (i.e. any string). This will involve vectorizing the list of documents, and classifying those vectors. Once that's complete, construct an inverse language mapping, from predicted indices to the language they represent.

In [25]:
def predict(
    documents: list[str],
    W: np.ndarray,
    feature_map: dict[str, int],
    lang_map: dict[str, np.ndarray],
    ngram_length: int,
    return_class_names=False,
):
    # your code here

    # print("Lang map", lang_map)
    # print("W in vectorize", W)

    lang_dict = {idx: lang for lang, onehot_vec in lang_map.items() for idx, vec in enumerate(onehot_vec) if vec == 1}
    # print("Lang dict", lang_dict, "\n")
    predictions = []
    for doc in documents:
        x = vectorize_document(doc, feature_map, ngram_length)
        # print("X in vectorize", x)
        lang_idx = classify(W, x)
        # print("Classify:", lang_idx)
        if return_class_names:
            predictions.append(lang_dict[lang_idx])
        else:
            predictions.append(lang_idx)

    # print("Predictions:", predictions)
    return predictions

Now, for each instance in the test set, print a tuple of the actual label, then the predicted label.

In [26]:
# the output of this cell will be used for test 3.1
test_labels, test_documents = zip(*test_observations)
test_predictions = predict(
    test_documents,
    W_inspect,
    feature_map,
    lang_map,
    ngram_length=INSPECTION_NGRAMS,
    return_class_names=True,
)

print(list(zip(test_labels, test_predictions)))

[('swahili', 'swahili'), ('indonesian', 'indonesian'), ('tagalog', 'tagalog'), ('indonesian', 'indonesian'), ('hausa', 'hausa'), ('nahuatl', 'nahuatl'), ('hausa', 'hausa'), ('manobo', 'manobo'), ('swahili', 'swahili'), ('swahili', 'swahili'), ('tagalog', 'tagalog'), ('indonesian', 'indonesian'), ('tagalog', 'tagalog'), ('tagalog', 'tagalog'), ('swahili', 'swahili'), ('swahili', 'swahili'), ('hausa', 'swahili'), ('indonesian', 'indonesian'), ('manobo', 'manobo'), ('tagalog', 'tagalog'), ('swahili', 'swahili'), ('swahili', 'swahili'), ('manobo', 'manobo'), ('swahili', 'swahili'), ('manobo', 'manobo'), ('swahili', 'swahili'), ('tagalog', 'tagalog'), ('swahili', 'swahili'), ('swahili', 'swahili'), ('nahuatl', 'nahuatl'), ('nahuatl', 'nahuatl'), ('swahili', 'swahili'), ('swahili', 'swahili'), ('swahili', 'swahili'), ('tagalog', 'tagalog'), ('indonesian', 'indonesian'), ('nahuatl', 'nahuatl'), ('tagalog', 'tagalog'), ('tagalog', 'tagalog'), ('tagalog', 'tagalog'), ('tagalog', 'tagalog'), ('t

### Plotting a Confusion Matrix

 A _confusion matrix_ is a $k \times k$ matrix, where k is your number of classes, where the cell in position $(i,j)$ counts the number of instances that belong to class $i$ that were predicted to be in class $j$. The diagonal entries represent correct classifications; anything off of the diagonal represents an incorrect classification. The example below, from the [scikit learn documentation](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.ConfusionMatrixDisplay.html), shows a confusion matrix for a binary classification problem.

![A confusion matrix for a binary classification problem](confusion_matrix.png)


Confusion matrices can be useful to see what types of errors your classifier is making. If errors are concentrated into particular cells, it could indicate the kind of data that your classifier struggles with. Write a function to take a list of test observations, and output a numpy array that represents your classifier's confusion matrix.

In [27]:
def get_confusion_matrix(
    labels: list[str], predictions: list[str], lang_map: dict[str, np.ndarray]
) -> np.ndarray:
    # your code here

    lang_list = sorted(lang_map.keys())

    lang_to_idx = {lang: idx for idx, lang in enumerate(lang_list)}
    # print("Lang indx", lang_to_idx)
    num_langs = len(lang_list)

    matrix = np.zeros((num_langs, num_langs), dtype=int)

    for true_label, pred_label in zip(labels, predictions):
        # print("True label", true_label)
        # print("Predicted label", pred_label)

        i = lang_to_idx[true_label]
        j = lang_to_idx[pred_label]

        matrix[i, j] += 1


    return matrix

Now, print your confusion matrix.

In [28]:
# the output of this cell will be used for test 3.2
get_confusion_matrix(test_labels, test_predictions, lang_map)

array([[ 18,   1,   0,   0,   9,   0],
       [  0,  31,   0,   0,   0,   1],
       [  0,   0,  32,   0,   1,   1],
       [  0,   0,   0,  35,   2,   0],
       [  0,   0,   0,   0, 148,   0],
       [  0,   0,   0,   0,   1, 120]])

From your confusion matrix, how many Hausa examples are misclassified as Swahili? Print each of the misclassified documents on a new line.

In [29]:
# the output of this cell will be used for test 3.3
# your code here
lang_list = sorted(lang_map.keys())
hausa_index = lang_list.index('hausa')
# swahili_index = lang_list.index('swahili')

misclassified_documents = []

# print("Test set:", test_set, "\n")

# print("Test observations", test_observations)

for index, (true_label_vec, doc_vector) in enumerate(test_set):

    true_label_index = np.argmax(true_label_vec)
    true_label = lang_list[true_label_index]

    hyp_label_index = classify(W_inspect, doc_vector)
    hyp_label = lang_list[hyp_label_index]

    if true_label == 'hausa' and hyp_label == 'swahili':

        misclassified_doc = test_observations[index][1]
        misclassified_documents.append(misclassified_doc)


for doc in misclassified_documents:
    print(doc)



print(f"\nNumber of Hausa examples misclassified as Swahili:", len(misclassified_documents))

Dalilin haka kuwa sun nuna cewa ayukan da shari'a take bukata na nan a rubuce a zuciyarsu. lamirinsu kuma na yi masu shaida, tunanainsu kuma, ko dai yana kashe su, ko kuma yana karesu.
Sai aka kawo kansa bisa tire, aka mika wa yarinyar, ta kuwa kai wa mahaifiyarta.
Baku sani ba mu za mu yiwa mala'iku shari'a? Balle shari'ar al'amuran wannan rai?
Da suka kai su Kotu, suka ce, ''Wadannan mutane Yahudawa ne, suna kawo tashin hankali a birninmu.
Ya ce masu, "A rubuce yake, cewa Almasihu za ya sha wuya, zai tashi kuma daga matattu a rana ta uku.
Bayan shekara uku na je Urushalima don in san Kefas, na zauna da shi na kwana goma sha biyar.
Ga wani yin ayyukan iko, ga wani kuwa annabci. Ga wani kuwa an ba shi baiwar bambance ruhohi, ga wani harsuna daban daban, kuma ga wani fassarar harsuna.
Amma da hankalinsa ya dawo, ya ce, 'Barorin mahaifina su nawa ne da suke da abinci isasshe, amma ina nan a nan, ina mutuwa sabili da yunwa!
Wannan dabban da na gani yana kama da damisa, kafafunsa kuma kama

Can you formulate a hypothesis for why these Hausa examples are being misclassified as Swahili? Peruse the Wikipedia pages of the two languages, and inspect the data and features of the two languages. Consider reasons based in what you know about the languages, and about machine learning. Try to come up with 2-3 experiments you might run to validate your hypothesis.

### Examining Feature Weights

The classifier that we've built for softmax classification behaves in many ways like using several binary softmax classifiers stacked together. The $K \times N$ weight matrix `W_inspect` (from the trigram classifier trained above) can be interpreted as a $1 \times N$ feature vector for each class, where each row represents the weights for one language class.

**Important: Use the `W_inspect` matrix from the "Inspecting Classification Results" section above for this analysis.**

In this section, we'll examine the weights for a few of our classified languages. To get feature names out of these vectors, we'll construct an inverted feature map, that maps indices in the feature vectors back to n-grams. Then, extract the $1 \times N$ vector that corresponds to Hausa (one row from `W_inspect`). Print the shape of the Hausa feature vector.

In [30]:
# the output of this cell will be used for test 4.1
# Invert the feature map
inverted_feature_map = {v: k for k, v in feature_map.items()}

# your code here

# Get the index for the Hausa language from the lang map
hausa_index = np.argmax(lang_map['hausa'])
# print("Hausa index", hausa_index)

# Get the feature vector for Hausa from the weight matrix W_inspect excluding the bias term
hausa_feature_vector = W_inspect[hausa_index, 1:] 

# print("Hausa feature vec", hausa_feature_vector)

print(hausa_feature_vector.shape)

(7814,)


Now, let's find the features that are most strongly predictive of an instance being Hausa. From the Hausa feature vector, find the names of the features with the top-10 positive values. Print your results with a tuple of (feature name, feature_weight) on each line:
```
("aaa", 0.04597442104739769)
("bbb", 0.03454984682736487)
```

In [31]:
# the output of this cell will be used for test 4.2
# your code here

hausa_feature_vector = W_inspect[hausa_index, 1:] 

# print("hausa_feature_vector", hausa_feature_vector, "\n")

top_indices_with_bias = np.argsort(hausa_feature_vector[1:])[-10:] # get the indices that would sort the array of feature vectors with bias

# print("Top 10 indices with bias", top_indices_with_bias, "\n")

top_indices_with_bias = top_indices_with_bias[::-1] # flip the indices to turn asxending to become descendng order

# print("Top 10 indices with bias desc", top_indices_with_bias, "\n")

top_10_indices = top_indices_with_bias + 1 # add 1 to account for the bias term that was removed


# get the features corresponding to the top 10 indices using the inverted feature map array
for i in top_10_indices:

    print((inverted_feature_map[i], float(hausa_feature_vector[i])))



('da ', 0.06451164031068245)
(' da', 0.05814918884378874)
('in ', 0.032426575762492936)
(' ba', 0.02643611391809732)
('ar ', 0.02555232352139255)
(' ya', 0.024015673618085424)
(' sh', 0.021499026046531342)
('ai ', 0.02050666010795031)
('ya ', 0.019389629427255823)
(' su', 0.018342154705746547)


Now, for each language, print the top-10 features. Print the name of each language, and then a list of it's top-10 features on the following line, e.g.

```
hausa
[("aaa", 0.04597442104739769)...]
indonesian
[("aaa", 0.04597442104739769)...]
```

In [32]:
# the output of this cell will be used for test 4.3
# your code here

lang_list = sorted(lang_map.keys())


for lang in lang_list:
    print(lang)
    
    lang_index = np.argmax(lang_map[lang])

    lang_weights = W_inspect[lang_index, 1:]

    # print("Lang weights", lang_weights)

    top_indices_with_bias = np.argsort(lang_weights[1:])[-10:] # get the indices that would sort the array of feature vectors with bias

    # print("Top 10 indices with bias", top_indices_with_bias, "\n")

    top_indices_with_bias = top_indices_with_bias[::-1] # flip the indices to turn asxending to become descendng order


    top_10_indices = top_indices_with_bias + 1 # add 1 to account for the bias term that was removed

    # print("Inverted feature map", inverted_feature_map)
    
    # get the features corresponding to the top 10 indices using the inverted feature map array
    top_10_features = [(inverted_feature_map[i], float(lang_weights[i])) for i in top_10_indices] 


    print(top_10_features)

hausa
[('da ', 0.06451164031068245), (' da', 0.05814918884378874), ('in ', 0.032426575762492936), (' ba', 0.02643611391809732), ('ar ', 0.02555232352139255), (' ya', 0.024015673618085424), (' sh', 0.021499026046531342), ('ai ', 0.02050666010795031), ('ya ', 0.019389629427255823), (' su', 0.018342154705746547)]
indonesian
[('an ', 0.06799812627407179), (' me', 0.06543762169208904), ('ang', 0.05864670458556179), (' se', 0.03851761535028479), ('ber', 0.0383095340146406), (' ke', 0.037826103731030895), (' be', 0.037697260417012685), ('men', 0.03458735237992753), (' di', 0.03337809762163509), ('ah ', 0.033316062263282056)]
manobo
[(' to', 0.11172142481570511), ('to ', 0.10867440466425177), (' no', 0.0703878240240312), ('no ', 0.05737575834850775), ('an ', 0.03610684793873696), (' og', 0.03465077972150404), ('n t', 0.03263541784142221), ('ow ', 0.031452540844881166), ('din', 0.029471839883121024), ('on ', 0.02843104526266351)]
nahuatl
[('tla', 0.06563090226122159), (' tl', 0.0623425085692276

Choose one of the languages, and peruse its Wikipedia page or other reliable resources to learn a bit more about the language. Do the top 10 features in that language make sense given what you've learned about the structure of the language? Why or why not? Again, consider reasons stemming both from what you've learned about the language, as well as machine learning and multinomial logistic regression specifically.

### Extra Credit

Implement at least one of the experiments you devised to test your hypothesis regarding misclassification of Hausa as Swahili. In order to get credit you must submit not just the code and results, but also clearly describe your hypothesis, the experiment, and why the experiment is suitable to test your hypothesis. Full credit will be given to hypotheses and experiments that are well thought out, explained clearly and convincingly, and backed up with suitable evidence (computed or cited).

In [43]:
# Implement extra credit here.

# Hypothesis:
# Hausa documents are misclassified as Swahili because they both originated frome arabic script origin and share a significant number of loanwords. These loan seen in common character ngrams. 

# Experiment:
# To test this, I will use trigrams and compare the most frequent trigrams in Hausa and Swahili training data to see how many overlap."]

# Implementation:
def get_top_ngrams(observations, language, n=3, top_k=20):
    lang_docs = [doc for lang, doc in observations if lang == language]
    all_ngrams = []
    for doc in lang_docs:
        all_ngrams.extend(extract_ngrams(doc, n))

    return Counter(all_ngrams).most_common(top_k)

swahili_top_ngrams = get_top_ngrams(train_observations, 'swahili')
hausa_top_ngrams = get_top_ngrams(train_observations, 'hausa')

# Analysis:
# The result shows that hausa and swahili have common character trigrams with the letter 'a' and a space appearing in most of them. This makes the model to assign almost the same weights to these trigrams which then makes the classifier to interpret hausa as swahili and vice versa
print("Top Swahili Trigrams:", swahili_top_ngrams)
print("Top Hausa Trigrams:", hausa_top_ngrams)


common_ngrams = set([ngram for ngram, count in hausa_top_ngrams]) & set([ngram for ngram, count in swahili_top_ngrams])
print("Common Top Trigrams:", common_ngrams)


Top Swahili Trigrams: [('wa ', 864), (' wa', 779), ('a k', 723), ('na ', 676), (' na', 545), ('a m', 534), (' ya', 473), (' ku', 458), ('ya ', 377), (' kw', 367), ('a n', 356), ('ali', 348), ('a w', 336), ('ka ', 326), ('ni ', 323), ('kwa', 310), ('aka', 286), (' ma', 252), (' ka', 241), ('ana', 238)]
Top Hausa Trigrams: [('da ', 188), (' da', 178), (' ya', 124), ('in ', 124), ('a k', 117), ('ya ', 112), ('an ', 112), (' ka', 107), ('ka ', 97), (' ba', 95), ('a s', 86), ('na ', 82), (' ma', 80), (' ku', 73), ('ma ', 72), ('ar ', 69), (' wa', 69), ('a y', 68), ('a a', 66), (' ta', 65)]
Common Top Trigrams: {' ya', ' ma', 'na ', 'a k', 'ya ', ' ku', ' wa', ' ka', 'ka '}
